In [ ]:
import sqlite3

# 1. DBへのコネクションを確立
DB_NAME = 'google_repos_data.db' # 確定したDBファイル名
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# 2. テーブルの定義とSQLの実行
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    language TEXT,
    stars INTEGER
)
"""
cursor.execute(CREATE_TABLE_SQL)

# 3. コミット（テーブル作成を確定）
conn.commit()

print(f"データベース '{DB_NAME}' に接続し、テーブル 'repos' を作成しました。")